# Copyright 2022 Cognite AS

## Import the Libraries and Modules

In [ ]:
import sys
from pathlib import Path

utils = str(Path("../utils").resolve())
if utils not in sys.path:
    sys.path.append(utils)

from cognite_auth import interactive_client

## Create the Cognite Client

In [ ]:
c = interactive_client()

## List operation

**Code Pattern**
`client.<cdf_resource_type>.list()`
where **cdf_resource_type** = { data_sets, assets, time_series, events, files, labels, sequences, relationships ... }


### List the CDF Resource types

In [ ]:
c.data_sets.list()

In [ ]:
c.assets.list(limit=5)

In [ ]:
df = c.time_series.list(limit=None).to_pandas()

In [ ]:
df.shape

In [ ]:
c.events.list(limit=5)

In [ ]:
c.files.list(limit=5)

Similar code for listing other resource types


```
c.labels.list()
```
```
c.sequences.list()
```
```
c.relationships.list()
```


In [ ]:
c.labels.list(limit=5)

In [ ]:
c.sequences.list(limit=5)

In [ ]:
c.relationships.list(limit=5)

### Filter the list results

using label filter, the data in cdf doesn't have these labels. But it is an example on how to filter with labels

In [ ]:
from cognite.client.data_classes import LabelFilter

In [ ]:
c.assets.list(labels=LabelFilter(contains_all=["industrial-assets"]),limit=5)

In [ ]:
c.assets.list(labels=LabelFilter(contains_any=["industrial-assets"]))

Other filters

In [ ]:
c.data_sets.list()

In [ ]:
c.data_sets.list(write_protected=True)

In [ ]:
c.assets.list(root=True)

### Iterate over the list

In [ ]:
for data_set in c.data_sets:
    print(data_set.id) # do something with the data_set

When List is too big, then use the **chunk_size** parameter to get the list in chunks

In [ ]:
for data_set_list in c.data_sets(chunk_size=5):
    print([x.name for x in data_set_list]) # do something with the list

## Searching in CDF

- Search the CDF resource types
- Filter the search results

**Code Pattern**

`client.<cdf_resource_type>.search()`

### Fuzzy Search on one field

In [ ]:
c.assets.search(name="Norway",limit=5)

In [ ]:
c.time_series.search(name="North",limit=5)

In [ ]:
c.files.search(name="sheet")

Similarly for sequences and events

`c.sequences.search(name="some name")`

`c.events.search(description="some description")`

### Multi-field fuzzy search

In [ ]:
c.assets.search(query="Nor",limit=5)

### Exact search on one field (e.g. name, description etc)

In [ ]:
c.assets.search(filter={"name": "Canada"})

get all timeseries for the above asset

In [ ]:
c.time_series.search(filter={"asset_ids":[8516698919785878]})

In [ ]:
c.events.list()[0].type

In [ ]:
# First get an example type to filter on
example_type = c.events.list()[0].type
# Filter the events of that type
c.events.search(filter={"type":example_type})

### Multiple filters in search function

In [ ]:
c.assets.list(limit=5,data_set_external_ids=['world_info_academy'])

In [ ]:
c.data_sets.list()

In [ ]:
c.assets.list(root=True)

In [ ]:
c.assets.search(name="San",filter={"parent_ids": [349354003007241]})

### Filter asset search using Label Filter

In [ ]:
from cognite.client.data_classes import AssetFilter

In [ ]:
c.assets.search(name="Norway",filter=AssetFilter(labels=LabelFilter(contains_all=["cold-countries"])),limit=5)

In [ ]:
c.assets.search(name="Can",filter=AssetFilter(labels=LabelFilter(contains_all=["cold-countries"])),limit=5)

## Retrieve the CDF resource types and data

### Code Pattern
`client.<cdf_resource_type>.retrieve()`

`client.<cdf_resource_type>.retrieve_multiple()`

### Retrieve the CDF resource types

single item - using id

In [ ]:
c.data_sets.list()

In [ ]:
# id of dataset
c.data_sets.retrieve(id=2238694842784134)

In [ ]:
c.time_series.retrieve(id=2238694842784134)

In [ ]:
import datetime
# id of timeseries
c.time_series.data.retrieve(id=658605450035112, start=datetime.datetime(2018,1,1), end=datetime.datetime(2018,2,1))

single item - using external_id

In [ ]:
c.data_sets.retrieve(external_id = "world_info_academy")

multiple items - using ids

In [ ]:
c.data_sets.list()

In [ ]:
c.data_sets.retrieve_multiple(ids=[39210236434027, 145208858652069])

multiple items - using external_ids

In [ ]:
c.data_sets.retrieve_multiple(external_ids=["test_dataset1", "test_dataset2","my_important_dataset"], ignore_unknown_ids=True)

In [ ]:
from datetime import datetime
# Don't put a long datetime range, as it'll fetch all the raw datapoints, which can be huge
c.time_series.data.retrieve(external_id=['Nauru_population','Spain_population'],start=datetime(2000,1,1),end=datetime(2024,1,2))

### Retrieve all items related to an asset

In [ ]:
# get the asset object for "Europa"
asset_obj = c.assets.retrieve(id = 349354003007241)

In [ ]:
asset_obj

Asset Subtree

In [ ]:
c.assets.retrieve_subtree(id = 349354003007241)

Children

In [ ]:
asset_obj.children()

Parent

In [ ]:
asset_obj.parent()

Events

In [ ]:
# get the asset object for "Italy"
asset_obj = c.assets.retrieve(id = 2689912833887590)

In [ ]:
asset_obj.events(limit=-1)

Timeseries

In [ ]:
# get the asset object for "Italy"
asset_obj = c.assets.retrieve(id = 2689912833887590)

In [ ]:
asset_obj.time_series()

### Retrieve data

#### Retrieve Raw Data

In [ ]:
# Don't put a long datetime range, as it'll fetch all the raw datapoints, which can be huge for Raw datapoints
c.time_series.data.retrieve(id=239507005016003, end='now')

#### Retrieve Aggregated data

In [ ]:
c.time_series.data.retrieve_dataframe(id=239507005016003,
                    end='1000w-ago',
                    aggregates=["average","sum"],
                    granularity="1h")

Retrieve latest datapoint before a particular time (last or any past time point)

In [ ]:
c.time_series.data.retrieve_latest(id=239507005016003, before="5w-ago")[0]

### Plot the data points

In [ ]:
c.time_series.data.retrieve(id=239507005016003, end='1000w-ago').to_pandas().plot()

### Download files

Download files to disk

In [ ]:
files_list = c.files.search(filter={"uploaded":True},limit=5)
files_list

In [ ]:
! mkdir files

In [ ]:
c.files.download(directory="files", id=[202614441188439, 4009508514690241])

Download a single file to a specific path

In [ ]:
c.files.download_to_path("files/test.svg", id=202614441188439)